In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir('/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/sil_consistent_at_inference')
print(os.getcwd())

/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/sil_consistent_at_inference


In [2]:
import glob
import random

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tqdm.autonotebook import tqdm

/opt/conda/envs/standard/lib/python3.6/site-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [17]:
# path to dir with shapenet pngs

input_img_dir = "data/input_images/pytorch3d_shapenet_renders_training/03001627/rgba"
out_dir = "data/input_images/pytorch3d_shapenet_renders_training/03001627/places"
places_dataset_dir = "/home/svcl-oowl/dataset/Places/val_256"

#input_img_dir = "data/input_images/pytorch3d_shapenet_renders/03001627/rgba"
#out_dir = "data/input_images/pytorch3d_shapenet_renders/03001627/places"
#places_dataset_dir = "/home/svcl-oowl/dataset/Places/test_256"

img_size = 224

os.makedirs(out_dir, exist_ok=True)
random.seed(0)

In [18]:
places_img_paths = glob.glob(os.path.join(places_dataset_dir, "*.jpg"))
all_original_img_paths = glob.glob(os.path.join(input_img_dir, "*.png"))

In [19]:
def place_bg(original_img_path):
    random_places_img_path = random.sample(places_img_paths, 1)[0]
    places_img = Image.open(random_places_img_path).resize((img_size, img_size))
    
    # opening rgba image but making the alpha mask binary
    original_img = Image.open(original_img_path)
    original_img_np = np.array(original_img)
    original_img_np[...,3] = (original_img_np[...,3]>0)*255
    original_img = Image.fromarray(original_img_np)
    
    # https://stackoverflow.com/questions/5324647/how-to-merge-a-transparent-png-image-with-another-image-using-pil
    places_img.paste(original_img, (0,0), original_img)
    return places_img

In [70]:

for original_img_path in tqdm(all_original_img_paths):
    random_places_img_path = random.sample(places_img_paths, 1)[0]
    places_img = Image.open(random_places_img_path).resize((img_size, img_size))
    
    # opening rgba image but making the alpha mask binary
    original_img = Image.open(original_img_path)
    original_img_np = np.array(original_img)
    original_img_np[...,3] = (original_img_np[...,3]>0)*255
    original_img = Image.fromarray(original_img_np)
    
    # https://stackoverflow.com/questions/5324647/how-to-merge-a-transparent-png-image-with-another-image-using-pil
    places_img.paste(original_img, (0,0), original_img)
    
    original_filename = original_img_path.split('/')[-1]
    # TODO: need to ensure output has 3 channels
    places_img.save(os.path.join(out_dir, original_filename))
    

In [21]:
# checking that everything is 3-channels
all_generated_img_paths = glob.glob(os.path.join(out_dir, "*.png"))
for generated_img_path in tqdm(all_generated_img_paths):
    img = Image.open(generated_img_path)
    if len(np.array(img).shape)!=3 or np.array(img).shape[0] != 224 or np.array(img).shape[1] != 224 or np.array(img).shape[2] !=3:
        print("Recreating Img")
        print(generated_img_path)
        instance = generated_img_path.split('/')[-1].split('.')[0]
        print(instance)
        print(np.array(img).shape)
        new_img = place_bg(os.path.join(input_img_dir, "{}.png".format(instance)))
        new_img.save(os.path.join(out_dir, "{}.png".format(instance)))